In [16]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from tifffile import imread, imwrite
import numpy as np
import pandas as pd
from skimage.draw import polygon
from pathlib import Path
from loguru import logger
from IPython.display import clear_output

from plex_pipe.utils.config_loaders import load_analysis_settings
from plex_pipe.core_cutting.channel_scanner import discover_channels, build_transfer_map
from plex_pipe.core_cutting.controller import CorePreparationController
from plex_pipe.core_cutting.file_io import GlobusFileStrategy
from plex_pipe.utils.globus_utils import GlobusConfig, create_globus_tc
from plex_pipe.utils.file_utils import GlobusPathConverter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# define pathways
settings_path = r'C:\sdata_toy_example\analysis_settings_LCB004-N-P_todel.yaml'

globus_config_path = r'D:\globus_config\globus_config.yaml'

### Load analysis settings

In [18]:
# load analysis configuration
settings = load_analysis_settings(settings_path)
settings

2025-11-19 19:06:37.098 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:51 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/LCB004-N-P/LCB004-N-P_Final', analysis_name='example_analysis', local_analysis_dir='C:/sdata_toy_example', remote_analysis_dir='/ix1/kkedziora', log_dir=None), core_detection=CoreDetectionSettings(detection_image='LCB004-N-P_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=None, use_markers=['DAPI', 'ECad'], ignore_markers=['Antibody1'], margin=0, mask_value=0, transfer_cleanup_enabled=False, core_cleanup_enabled=False), additional_elements=[NormalizeStep(category='image_transformer', type='normalize', input='DAPI', output='DAPI_norm', keep=False, parameters=Params(low=1.0, high=99.8)), NormalizeStep(category='image_transformer', type='normalize', input='ECad', output='ECad_norm', keep=False, parameters=Params(

### Define the logger

In [6]:
log_file = settings.log_dir_path / f"cores_cutting_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

### Set up Globus

In [7]:
# get globus config
gc = GlobusConfig.from_config_files(globus_config_path, from_collection = 'r_collection_id', to_collection = 'cbi_collection_id')
tc = create_globus_tc(gc.client_id, gc.transfer_tokens)

In [11]:
# if Windows paths change to Globus
image_path = settings.general.image_dir
if ":/" in settings.general.image_dir or ":\\" in settings.general.image_dir:
    conv = GlobusPathConverter(layout="single_drive")
    image_path = conv.windows_to_globus(image_path)

image_path

'/CellDive/LCB004-N-P/LCB004-N-P_Final'

In [12]:
channel_map = discover_channels(image_path,
                                include_channels=settings.core_cutting.include_channels,
                                exclude_channels=settings.core_cutting.exclude_channels,
                                use_markers=settings.core_cutting.use_markers,
                                ignore_markers=settings.core_cutting.ignore_markers,
                                gc=gc)

2025-11-19 16:30:57.674 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:77 - Discovered 19 channels:
2025-11-19 16:30:57.674 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:79 - 001_AP2B <- /CellDive/LCB004-N-P/LCB004-N-P_Final/LCB004-N-P_1.0.4_R000_Cy5_AP2B-AF647_FINAL_AFR_F.ome.tif
2025-11-19 16:30:57.674 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:79 - 001_DAPI <- /CellDive/LCB004-N-P/LCB004-N-P_Final/LCB004-N-P_1.0.4_R000_DAPI__FINAL_F.ome.tif
2025-11-19 16:30:57.674 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:79 - 001_FOXA1 <- /CellDive/LCB004-N-P/LCB004-N-P_Final/LCB004-N-P_1.0.4_R000_Cy3_FOXA1-AF555_FINAL_AFR_F.ome.tif
2025-11-19 16:30:57.674 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:79 - 002_DAPI <- /CellDive/LCB004-N-P/LCB004-N-P_Final/LCB004-N-P_2.0.4_R000_DAPI__FINAL_F.ome.tif
2025-11-19 16:30:57.674 | INFO     | plex_pipe.

In [23]:
channel_map

{'DAPI': '/CellDive/LCB004-N-P/LCB004-N-P_Final/LCB004-N-P_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'ECad': '/CellDive/LCB004-N-P/LCB004-N-P_Final/LCB004-N-P_4.0.4_R000_Cy3_ECad-AF555_FINAL_AFR_F.ome.tif'}

In [21]:
df_path = settings.core_info_file_path.with_suffix('.pkl')

df = pd.read_pickle(df_path)
df.head()

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,14208.0,14784.0,70656.0,71296.0,rectangle,"[[14784.0, 71296.0], [14784.0, 70656.0], [1420..."
1,Core_001,13888.0,14528.0,69376.0,70080.0,rectangle,"[[13888.0, 70080.0], [14528.0, 70080.0], [1452..."


In [14]:
# build transfer map
transfer_cache_dir = settings.temp_dir
transfer_map = build_transfer_map(channel_map, transfer_cache_dir)

# build a dict for transfered images
image_paths = {
    ch: str(Path(transfer_cache_dir) / Path(remote).name)
    for ch, (remote, _) in transfer_map.items()
}
image_paths

{'DAPI': 'C:\\sdata_toy_example\\example_analysis\\temp\\LCB004-N-P_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'ECad': 'C:\\sdata_toy_example\\example_analysis\\temp\\LCB004-N-P_4.0.4_R000_Cy3_ECad-AF555_FINAL_AFR_F.ome.tif'}

In [15]:
transfer_map

{'DAPI': ('/CellDive/LCB004-N-P/LCB004-N-P_Final/LCB004-N-P_1.0.4_R000_DAPI__FINAL_F.ome.tif',
  '/C/sdata_toy_example/example_analysis/temp/LCB004-N-P_1.0.4_R000_DAPI__FINAL_F.ome.tif'),
 'ECad': ('/CellDive/LCB004-N-P/LCB004-N-P_Final/LCB004-N-P_4.0.4_R000_Cy3_ECad-AF555_FINAL_AFR_F.ome.tif',
  '/C/sdata_toy_example/example_analysis/temp/LCB004-N-P_4.0.4_R000_Cy3_ECad-AF555_FINAL_AFR_F.ome.tif')}

In [15]:
settings

AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/NSR7649/NSR7649_Final', analysis_name='NSR7649_Analysis', local_analysis_dir='C:/sdata_NSR7649', remote_analysis_dir='/ix1/kkedziora', log_dir=None), core_detection=CoreDetectionSettings(detection_image='NSR7649_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=None, use_markers=None, ignore_markers=['Antibody1'], margin=0, mask_value=0, transfer_cleanup_enabled=True, core_cleanup_enabled=True), additional_elements=[NormalizeStep(category='image_transformer', type='normalize', input='DAPI', output='DAPI_norm', keep=False, parameters=Params(low=1.0, high=99.8)), NormalizeStep(category='image_transformer', type='normalize', input='ECad', output='ECad_norm', keep=False, parameters=Params(low=1.0, high=99.8)), Insta

In [ ]:
strategy = GlobusFileStrategy(tc=tc, transfer_map=transfer_map, gc=gc, cleanup_enabled = True) # submits transfers when initialized, should it stay like this?

controller = CorePreparationController(
    metadata_df = df, # df defines which cores to process
    image_paths = channel_map, # defines which channels to use
    temp_dir = settings.cores_dir_tif_path,
    output_dir = settings.cores_dir_output_path,
    file_strategy = strategy,
    margin = settings.core_cutting.margin,
    mask_value = settings.core_cutting.mask_value,
    max_pyramid_levels = settings.sdata_storage.max_pyramid_level,
    chunk_size = settings.sdata_storage.chunk_size,
    downscale = settings.sdata_storage.downscale,
)

controller.run()

2025-10-15 14:56:12.519 | INFO     | plex_pipe.core_cutting.file_io:submit_all_transfers:91 - Submitted transfer for DAPI to /C/BLCA-2_Analysis_todel/temp/BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif (task_id=9d4b7100-a9f8-11f0-812b-0e092d85c59b)
2025-10-15 14:56:12.522 | INFO     | plex_pipe.core_cutting.controller:run:117 - Starting controller run loop...
2025-10-15 14:57:23.006 | INFO     | plex_pipe.core_cutting.file_io:is_channel_ready:16 - Transfer for DAPI complete: /C/BLCA-2_Analysis_todel/temp/BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif
2025-10-15 14:57:24.329 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:93 - Cut and saved core Core_001, channel DAPI
2025-10-15 14:57:24.332 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:100 - Closed file handle for channel DAPI
2025-10-15 14:57:25.102 | INFO     | plex_pipe.core_cutting.file_io:cleanup:212 - Cleaned up file: C:\BLCA-2_Analysis_todel\temp\BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif
2025-10-15 14:57:25.103 | INFO   